# CS110 - Final Project Implementation
## Burrows-Wheeler Transform 

In [1]:
import heapq
from collections import defaultdict
from bitarray import bitarray
import pickle
import time
import random, string

## BWT Transform Class

In [2]:
class BWT:
    def __init__(self, file):
        self.file = file
            
    def encode(self):
        """
        This function applies BWT on the string. Input a string and output 
        the transformed string together with the index of the original string.
        """
        rotations = BWT.create_rotations(list(self.file))
        sorted_rotations = sorted(rotations)
        encoded_string = []
        for i in sorted_rotations:
            encoded_string.append(i[-1]) #append the last character
        return (''.join(encoded_string), sorted_rotations.index(list(self.file)))

    def create_rotations(strlist):
        """
        This function create rotations of string S. Input a list of string and
        output a array contain rotations of that string. 
        """
        rotations_array = []
        str_len = len(strlist)
        for i in range(str_len):
            rotations_array.append(strlist[(str_len-i):]+strlist[:(str_len-i)])
        return rotations_array

    def decode(self):
        """
        Reverse_transformed BWT output into the original string.
        """
        L = self.file[0] #the transformed string 
        I = self.file[1] #index of the original string
        n = len(L)
        table = [[]]*n #create empty table
        for x in range(n):
            for y in range(n):
                table[y] = list(L[y]) + table[y]
            table = sorted(table)
        return ''.join(table[self.file[1]])

In [3]:
BWT("this is a great class.").encode()
BWT(('stass le r t hcgiisaa.', 21)).decode()

'this is a great class.'

In [4]:
def MTF(string):
    """
    Move-to-front encoding, input a text and return the encoded 
    text and the dictionary.     
    The implementation is referenced from: https://en.wikipedia.org/wiki/Move-to-front_transform
    """
    # Initialise the list of characters (i.e. the dictionary)
    dictionary = sorted(list(set(string)))
    # Transformation
    compressedtext = list()
    rank = 0
    # read in each character
    for c in string:
            rank = dictionary.index(str(c)) # find the rank of the character in the dictionary
            compressedtext += [str(rank)] # update the encoded text
            # update the dictionary
            dictionary.pop(rank)
            dictionary.insert(0, c)

    dictionary.sort() # sort dictionary
    return [compressedtext,dictionary]


In [46]:
MTF('stass le r t hcgiisaa.')

[['10',
  '11',
  '4',
  '2',
  '0',
  '3',
  '10',
  '7',
  '2',
  '11',
  '1',
  '6',
  '1',
  '10',
  '9',
  '10',
  '11',
  '0',
  '9',
  '10',
  '0',
  '11'],
 [' ', '.', 'a', 'c', 'e', 'g', 'h', 'i', 'l', 'r', 's', 't']]

## Huffman Compression

In [5]:
def build_freq(file):
    """
    Build the dictionary of frequency used for huffman encoding
    """
    freq = defaultdict(int)
    for char in file:
        freq[char] += 1
    total = float(sum(freq.values()))
    return {char: count / total for (char, count) in freq.items()}

# Now build the Huffman encoding:
def huffman_encode(frequency):
    """
    Huffman encodes the given dict mapping symbols to weights.
    Accept a dictionary which maps a symbol to a probability.
    Return a new dictionary which maps a symbol to a bitarray.    
    """
    heap = [[weight, [symbol, '']] for symbol, weight in list(frequency.items())]
    heapq.heapify(heap)
    while len(heap) > 1:
        low = heapq.heappop(heap)
        high = heapq.heappop(heap)
        for pair in low[1:]:
            pair[1] = '0' + pair[1]
        for pair in high[1:]:
            pair[1] = '1' + pair[1]
        heapq.heappush(heap, [low[0] + high[0]] + low[1:] + high[1:])
    return dict(sorted(heapq.heappop(heap)[1:], key=lambda p: (len(p[-1]), p)))
    
def huffman_compress(filename, encoding, compressed_name=None):
    """
    Compress the file using Huffman codes. Input filename and huffman codes 
    of the file, output file.huff.
    """
    if compressed_name is None:
        compressed_name = filename.rstrip('.txt') + ".huff"
    output = bitarray()
    with open(filename, 'r') as f:
        for line in f:
            for char in line:
                output.extend(encoding[char])
    N = len(output)
    with open(compressed_name, 'wb') as f:
        pickle.dump(N, f)
        pickle.dump(encoding, f)
        output.tofile(f)
        
def huffman_decompress(filename, decompressed_name=None):
    """
    Decompress .huff file.
    """
    if decompressed_name is None:
        decompressed_name = filename + ".dehuff"
    with open(filename, 'rb') as f:
        N = pickle.load(f)
        encoding = pickle.load(f)
        bits = bitarray()
        bits.fromfile(f)
        bits = bits[:N]

    # Totally cheating here and using a builtin method:
    output = bits.decode(encoding)

    output = "".join(output).encode('utf-8-sig')
    with open(decompressed_name, 'wb') as f:
        f.write(output)

def direct_huffman_compress(filename):
    """
    Directly compress the file using huffman code. The function builds the Huffman codes 
    from the file and encodes it. 
    """
    with open(filename, "r") as my_file:
        file = my_file.read()

    freq = build_freq(file)
    encoding = huffman_encode(freq)
    huffman_compress(filename, encoding)

direct_huffman_compress("Shakespeare.txt")

## Compressing with BWT

In [6]:
def compress(filename, string, huff_codes, char_set, ind):
    """
    Compress the MTF encoded file. 
    
    Input:
    filename (str): name of the file.
    string (str): MTF-encoded version of the file. 
    huff_codes (dict): Huffman codes built from the file. 
    char_set (list): list containing unique characters from the file. 
    ind (int): index of the original string from BWT     
    """
    
    fname = filename.rstrip('.txt') + '_BWT.huff'
    output = bitarray()
    for char in string:
        output.extend(huff_codes[char])
    N = len(output)
    with open(fname, 'wb') as f:
        pickle.dump(N, f)
        pickle.dump(huff_codes, f)
        pickle.dump(char_set, f)
        pickle.dump(ind, f)
        output.tofile(f)
        
def BWT_compress(filename):
    """
    Compress file using BWT transformed and MTF encoding before Huffman compress. 
    """
    with open(filename, "r") as my_file:
        file = my_file.read()
    BWT_transformed = BWT(file).encode()
    MTF_encoded = MTF(BWT_transformed[0])
    
    frequency = build_freq(MTF_encoded[0])
    huff_codes = huffman_encode(frequency)
    for key, value in huff_codes.items():
        huff_codes[key] = bitarray(value)
        
    compress(filename, MTF_encoded[0], huff_codes, MTF_encoded[1], BWT_transformed[1])

## Compare compression methods

In [7]:
def compare_compression(filename):
    print("Compressing", filename)
    start_time = time.time()
    direct_huffman_compress(filename)
    print("Compressing with normal Huffman takes {} seconds".format(time.time()-start_time))

    start_time = time.time()
    BWT_compress(filename)   
    print("Compressing with BWT takes {} seconds".format(time.time()-start_time))
    print("====================================================================")

## Create random test strings

In [ ]:
random_string = ''.join(random.choice(string.ascii_lowercase) for _ in range(50000))
textfile = open('random_string_test.txt', 'w')
textfile.write(random_string)
textfile.close()

In [12]:
int_sequence = ''.join(random.choice([str(0),str(1),str(2),str(3),str(4),str(5),str(6),str(7),\
                    str(8),str(9),str(10)]) for _ in range(10000))

textfile = open('Integers.txt', 'w')
textfile.write(int_sequence)
textfile.close()

In [ ]:
compare_compression("Shakespeare.txt")
compare_compression("Shakespeare_small.txt")
compare_compression("random_string_test.txt")
compare_compression("Harry_potter.txt")
compare_compression("Integers.txt")